# Practicing Good Enough Automation & Testing Practices

Building on our [Automating Your Build With Jenkins](09%20-%20Automating%20Your%20Build%20With%20Jenkins.ipynb) and [Automating Your Benchmarks With Jenkins](10%20-%20Automating%20Your%20Benchmarks%20With%20Jenkins.ipynb) notebooks, we will introduce some best practices to help us improve the reliability, scalability, performance, and visibility of our digital research and development cycle. 



# Everything as Code
Every aspect of your software lifecycle should be stored, versioned, and managed as application code. This includes infrastructure code, build scripts and pipeline code, documentation, etc. This practice is referred to as _Everything as Code_, and offers unrivaled visibility into your full software lifecycle and architecture.

Keeping everything in VCS provides a mechanism to reproduce your entire application at a particular moment in time, and makes sure that there is a single source of truth defining deployment, build, and execution behavior. Employing this practice allows all components of your application to be managed with the same rigor and consistency as your application code.

## Managing third-party code
Maintaining Everything as Code comes with challenges, particularly in projects where you don't control 100% of your code. Open science is built on open-source software, which means your application will likely depend on a number of third-party software and APIs. Forking all of your dependencies is unmaintainable, and ultimately subverts the open-source workflow because it becomes difficult or impossible to contribute back upstream.

A good strategy for managing third party code in this context comes from a mixture of the following guidelines:
#### Explicit Version Requirements
When listing dependencies, or packaging them with your application in a container build spec, be as specific as reasonably possible about which versions you are pulling down. A good rule of thumb is to be specific down to the minor version for third party APIs, and always make sure to avoid using the `:latest` tag for Docker/Singularity base images.
#### Meaningful Tagging Schemata
When tagging new releases of your application, make sure to treat dependency changes as impetuses for version changes, paying close attention to issues of backwards-comptaibility.
#### Manage Source Dependencies as Submodules
If it is not viable to manage a particular dependency with a package manager like `yum` or `pip`, it is often a successful approach to manage those dependencies as [submodules](https://git-scm.com/book/en/v2/Git-Tools-Submodules) in your application repository. The `git submodule` features allow for specific commit requirements to be maintained, don't require forking, and facilitate good open-source practices.
#### Use a Single Application Repository
Particularly in the case where you are utilizing submodules, your entire application lifecycle _(build and deploy pipelines, tests, application code, documentation)_ should be maintained in a single repository. This emphasizes the value of having a single source of truth, allows for the entire project to be tagged with a descriptive version, and allows for review and consistency across the entire codebase without a lot of effort placed into searching through disparate components.

## Branching, Versioning, Tagging Strategies

### GitFlow
[GitFlow](https://datasift.github.io/gitflow/IntroducingGitFlow.html) is a successful git branching workflow for teams of any size.

![The GitFlow Strategy](https://datasift.github.io/gitflow/GitFlowHotfixBranch.png)

A few suites of tools exist around GitFlow to help you adhere to community developed best practices surrounding versioning, publishing, collaboration, documentation, and reproduciblity of software.

* [git-flow](https://github.com/petervanderdoes/gitflow-avh): a command line tool to provide high-level repository operations for the GitFlow branching model.
* [git-flow-hooks](https://github.com/jaspernbrouwer/git-flow-hooks): a set of useful hooks to automate versioning, ensure proper branching behavior, and prepare your repository for release.

# Strategies for Maintaining Credentials and Config
Most build, test, and deploy pipelines require access to secret information and site-specific configuration to execute. When managing your entire application as code, and especially when it's all aggregated in a single repository, it is important to maintain a sensible state-of-practice for these credentials and configurations.

Best practice strategies for handling this kind of configuration in pipelines involves the use of a secret store, such as [HashiCorp Vault](https://www.vaultproject.io/) or [KeyWhiz](https://square.github.io/keywhiz/). These secret stores provide mechanisms for secure access and storage of sensitive information separate from your repository, and tie nicely into automation platforms like Jenkins.

If you're using a cloud solution like [TravisCI](https://travis-ci.com/) for automation, there are methods for [encrypting secrets](https://docs.travis-ci.com/user/encryption-keys/) to be stored in your application repository, and accessed by the automation platform during build.

# Continuous Delivery and Deployment
Once your build pipeline has run, and your changeset has passed all testing guards, what do you do? There are two common extension to Continuous Integration: _Continuous Delivery_ and _Continuous Deployment_.

## Continuous Delivery
Continuous Delivery automates the process of packaging and releasing code every time a passing commit is made to the production or release branch. Nightly builds are a good example of continuous delivery. Modern delivery processes often terminate in a container registry, like [DockerHub](https://hub.docker.com/) or [SingularityHub](https://singularity-hub.org/).

After a successful build, the delivery pipeline will take the new container, and upload it to the appropriate registry and tag it with a new version. This is excellent practice if your software is meant to be distributed and run by other users.

## Continuous Deployment
Continuous Deployment automates the process of deploying your software to production infrastructure, and often deploying and provisioning the infrastructure itself. This type of pipeline continuation necessitates further layers of testing, and is most frequently seen in \*-as-a-Service architectures which deploy new releases multiple times per day.

This is a good practice if you are deploying software to your own infrastructure that will be used by others.

# Testing
Without a strong testing culture, effort placed into software lifecycle automation will end in catastrophe. The following is a brief overview of testing best practices.

## Types of Tests
* **Unit:** A test of an individual unit of code, in isolation from other units.
* **Integration:** A test of multiple units interoperating with one another.
* **System:** Validation of an entire, integrated system against requirements.
* **Benchmarks:** A test that focuses on the performance characteristics of a unit or system over its correctness.

## Best Practices for Software Testing
Tests can be difficult and expensive to write and maintain, and a brittle or misleading test can be worse than no test at all. Below are a few guidelines that will help you write tests that maximize value while minimizing test volume. Many of these guidelines have been adapted from [Working Effectively With Unit Tests](https://leanpub.com/wewut/read) by Jay Fields.

### Write Tests for Readability
Tests are read more often than they are written, and need to be written in a way that reflects this reality. A test will be read when it fails, when the running application fails and the test didn't, or when a developer wants to add a backwards-compatible change to the feature a test validates. In all of these situations, the test communicates an assumption about the code's behavior that needs to be validated.

#### DAMP, not DRY
In the same way that **D**on't **R**epeat **Y**ourself (DRY) principles improve the orthogonality of a codebase, **D**escriptive **A**nd **M**eaningful **P**hrases (DAMP) principles can improve its readability. When writing tests, don't worry about de-duplicating code and procedures, instead focus on placing everything a reader needs to understand your test into the test itself. By DRYing out your test and breaking fixtures out into reusable components that are not colocated, you've increased the cognitive effort needed to derive value from your test. A great discussion of this can be found in this [Stack Overflow answer](https://stackoverflow.com/a/11837973).

#### Expect Literals
When possible, an assertion should compare output to literal values, and not objects or variables containing values. Helper functions that generate values to compare should also be avoided in favor of a literal value written straight into the test. Doing so makes it significantly easier for a reader of your test to understand what the test is expecting, and examine how the behavior of the code may have diverged from that expectation, without having to search through and understand ancillary logic.

#### One Assertion Per test
Each test should be written, when possible, to include a single assertion at the very end. This practice has a positive impact on readability, and prevents the first failed assertion in a test from halting execution and obscuring the success or failure of subsequent assertions. A single assertion per test also tends to improve the durability of your tests, as it divides your validation into behavioral facets of a unit of code. A small change to some facet no longer necessitates that all tests surrounding that unit be rewritten.

### Maximize Test Value
For codebases of any non-trivial size it is rarely viable to achieve 100% coverage, so one must be deliberate about which tests they write, and more importantly, which tests are maintained long-term. The following heuristics can help with this selection process.

#### Write Tests for Risk Factors
When prioiritizing test coverage at the application-level, focus first primary drivers of risk _(i.e., numerical errors or double-charging in an ecommerce application)_ and critical use cases _(i.e., adding to one's shopping cart)_. When prioritizing coverage on a class- or component-level, prioritize public interfaces.

#### Eliminate Low-Value Tests
If a test costs more to maintain than the cost it saves in discovering defects and validating changes, delete it.